Code for the testing and development of group_linear_regression_boot_strap

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from janelia_core.stats.regression import grouped_linear_regression_boot_strap

## Parameters go here

In [56]:
# Coefficients of the true model here
coefs = np.asarray([1.0, 3.0, 0.0, 4.0, 5.0])

# True model mean here
mn = 7

# Number of groups to sample
n_grps = 1000

# Range of the number of samples to generate from each group
n_smps_per_group = [1, 5]


## Generate true data

In [57]:
n_coefs = len(coefs)

x_grp = [None]*n_grps
y_grp = [None]*n_grps
i_grp = [None]*n_grps

for g_i in range(n_grps):
    n_grp_smps = np.random.randint(n_smps_per_group[0], n_smps_per_group[1] + 1)
    
    
    x_base = np.random.randn(1, n_coefs)
    x_add = .1*np.random.randn(n_grp_smps, n_coefs)
    x_grp[g_i] = x_base + x_add
    
    y_grp[g_i] = np.matmul(x_grp[g_i], coefs) + np.random.randn(n_grp_smps) + mn

    i_grp[g_i] = np.ones(n_grp_smps)*g_i
    
x = np.concatenate(x_grp)
y = np.concatenate(y_grp)
grps = np.concatenate(i_grp)

In [58]:
grp_inds = grouped_linear_regression_boot_strap(x=x, y=y, g=grps, n_bs_smps=10000)

In [59]:
np.mean(grp_inds, axis=0)

array([ 9.98713930e-01,  3.00631931e+00, -1.16847958e-03,  3.99188397e+00,
        4.98725898e+00,  6.97968932e+00])

In [60]:
np.std(grp_inds, axis=0)

array([0.01705191, 0.01797086, 0.01905397, 0.01780462, 0.01788956,
       0.0182089 ])